In [51]:
import os
import cv2
from codecs import open as copen
from shutil import copyfile, rmtree

from sklearn.model_selection import train_test_split
import requests
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [73]:
dms_url = 'dms.ei.team'
vision_url = 'api.vis.ei.team'
api_key_dms = 'U2sl0VOF.d9Ywe5lo7GYjUoeSSPsXKTB9fVdlIFO1'
api_key_vision = '1PmnQK4u.m1b2mA1QmIsVRNOq3eErQX1jtZh3Zrxu'
obj_path = './train/annotate_vision'

In [75]:
os.mkdir(obj_path)

In [35]:
images = requests.get('http://{}/v1/apikey/annotation'.format(vision_url),
                    headers={
                        "Authorization": "Api-Key {}".format(api_key_vision)
                    }
                   )
print(images)
images = images.json()['annotations']
print(images)

<Response [200]>
[{'image_uid': 121516, 'markers': [{'type': 'polygon', 'labels': ['plate'], 'coordinates': [{'x': 888, 'y': 1140}, {'x': 989, 'y': 1143}, {'x': 983, 'y': 1187}, {'x': 888, 'y': 1184}]}], 'labeler': 'SothanaV', 'reviewer': 'SothanaV', 'is_second_edit': False}, {'image_uid': 121506, 'markers': [{'type': 'polygon', 'labels': ['plate'], 'coordinates': [{'x': 1714, 'y': 1058}, {'x': 1796, 'y': 1061}, {'x': 1799, 'y': 1092}, {'x': 1714, 'y': 1092}]}], 'labeler': 'SothanaV', 'reviewer': 'SothanaV', 'is_second_edit': False}, {'image_uid': 121488, 'markers': [{'type': 'polygon', 'labels': ['plate'], 'coordinates': [{'x': 1751, 'y': 1070}, {'x': 2000, 'y': 1055}, {'x': 2007, 'y': 1143}, {'x': 1755, 'y': 1162}]}], 'labeler': 'SothanaV', 'reviewer': 'SothanaV', 'is_second_edit': False}, {'image_uid': 121456, 'markers': [{'type': 'polygon', 'labels': ['plate'], 'coordinates': [{'x': 1663, 'y': 894}, {'x': 1736, 'y': 891}, {'x': 1739, 'y': 922}, {'x': 1666, 'y': 925}]}], 'labeler': 

In [77]:
for i, image in enumerate(images):
    print(i+1,len(images))
    labels = []
    labelled = image['markers']
    url = requests.get('http://{}/v1/image/{}'.format(dms_url,image['image_uid'])).json()['file']
    req = urllib.request.urlopen(url)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    frame = cv2.imdecode(arr, -1)
    f_name = image['image_uid']
    heigth,weidth = frame.shape[:2]
#     print("w:{} h:{}".format(weidth,heigth))
#     plt.imshow(frame)
#     plt.show()
    txt=""
    obj_img_path = os.path.join(obj_path, "{}.png".format(f_name))
    obj_txt_path = os.path.join(obj_path, "{}.txt".format(f_name))
    for label in labelled:
        if label['type'] == 'polygon':
            coordinates = label['coordinates']
            m = np.zeros((4,2))
            for index,cor in enumerate(coordinates):
                m[index][0] = cor['x']/weidth
                m[index][1] = cor['y']/heigth
#             print(m)
            txt = "{},{},{},{},{},{},{},{},{}".format(4,
                    m[0][0],m[1][0],
                    m[2][0],m[3][0],
                    m[0][1],m[1][1],
                    m[2][1],m[3][1])
            print(txt)
    with open(obj_txt_path, 'w+') as fp:
        fp.write(txt)
        fp.write(os.linesep)
    cv2.imwrite(obj_img_path,frame)

1 10
4,0.22023809523809523,0.2452876984126984,0.24379960317460317,0.22023809523809523,0.5816326530612245,0.5831632653061225,0.6056122448979592,0.6040816326530613
2 10
4,0.42509920634920634,0.44543650793650796,0.4461805555555556,0.42509920634920634,0.539795918367347,0.5413265306122449,0.5571428571428572,0.5571428571428572
3 10
4,0.43427579365079366,0.49603174603174605,0.49776785714285715,0.43526785714285715,0.5459183673469388,0.5382653061224489,0.5831632653061225,0.5928571428571429
4 10
4,0.4124503968253968,0.4305555555555556,0.4312996031746032,0.4131944444444444,0.4561224489795918,0.4545918367346939,0.4704081632653061,0.4719387755102041
5 10
4,0.2906746031746032,0.31101190476190477,0.3117559523809524,0.29216269841269843,0.42244897959183675,0.41581632653061223,0.4336734693877551,0.43979591836734694
6 10
4,0.566468253968254,0.5959821428571429,0.597718253968254,0.5679563492063492,0.4785714285714286,0.47551020408163264,0.5010204081632653,0.5040816326530613
7 10
4,0.5124007936507936,0.53596